In [ ]:
from sklearn.preprocessing import MinMaxScaler
import itertools
import pandas as pd
import torch

# get our predefined functions (for data processing, etc.)
import QCircNet.utils as ut

# get our predefined quantum circuits and neural networks
import QCircNet.circuits.CustomCircuits as cn
import QCircNet.BinaryQuantumNN as qnn


EXPERIMENT_ID = 0
HYBRID = False

X_PATH = "QCircNet/data/X_apd_class6_7.npy"
Y_PATH = "QCircNet/data/y_apd_class6_7.npy"
BIN_ENCODING = {6: 0, 7: 1}


N_QUBITS = 4
FEAT_PER_QUBIT = 4

SCALER = MinMaxScaler()

SUBSET = None

EPOCHS = 10
WEIGHT_PATH = f"results/weights/bin/"
MODEL_PATH = f"results/models/bin/"
RESULTS_PATH = f"results/bin_model_results_expID_{EXPERIMENT_ID}.csv"
RESULTS_DF = pd.DataFrame()

# for printing colored text output during training
RED_TEXT = "\033[91m"
GREEN_TEXT = "\033[92m"
RESET_COLOR = "\033[0m" 

In [3]:
# load the data
X_train, X_val, X_test, y_train, y_val, y_test, _ = ut.load_and_prepare_data(X_PATH, Y_PATH, scaler=SCALER, bin_encoding=BIN_ENCODING, subset=SUBSET)

# check shapes
print("train:\t", X_train.shape, y_train.shape)
print("val:\t", X_val.shape, y_val.shape)
print("test:\t", X_test.shape, y_test.shape)

train:	 torch.Size([9746, 16]) torch.Size([9746])
val:	 torch.Size([1219, 16]) torch.Size([1219])
test:	 torch.Size([1218, 16]) torch.Size([1218])


In [4]:
cn.get_custom_circuits()

[QCircNet.circuits.CustomCircuits.Anusha2024Circuit,
 QCircNet.circuits.CustomCircuits.DoubleEntanglementVLCircuit,
 QCircNet.circuits.CustomCircuits.Ranga2024Circuit,
 QCircNet.circuits.CustomCircuits.Senokosov2024Circuit]

In [ ]:
circuits = [
    cn.Ranga2024Circuit,
    cn.Senokosov2024Circuit,
    cn.Anusha2024Circuit,
    cn.DoubleEntanglementVLCircuit,
]

runs = list(range(1, 11)) # runs are also used as seeds (reproducibility)
learning_rates = [0.01, 0.001]

In [6]:
### RUN EXPERIMENT
id = 0
for seed, circuit, lr in itertools.product(runs, circuits, learning_rates):
    print(f"""{RED_TEXT}RUN {id+1}:{RESET_COLOR}\n {circuit.__name__} \n lr: {lr}""")

    # init the model
    model = qnn.BinQuantumNeuralNetwork(n_qubits=N_QUBITS, features_per_qubit=FEAT_PER_QUBIT, circuit=circuit, seed=seed, hybrid=HYBRID)

    path_identifier = f"{circuit.__name__}_expID_{EXPERIMENT_ID}_dfID_{id}"

    # train the model (predifined optimizer, etc.)
    trained_model, losses, val_losses, val_f1s, avg_epoch_time = ut.train_model(
        model, X_train, y_train, X_val, y_val,
        epochs=EPOCHS, batch_size=32, lr=lr, binary=True,
    )
    
    # save trained model
    model_path = MODEL_PATH+path_identifier+".pth"
    torch.save(trained_model, model_path)

    # evaluate
    cross_entropy, accuracy, precision, recall, f1 = trained_model.evaluate(X_test, y_test)

    results_dict = {
        "id": id,
        "f1_score": f1, "precision": precision, "recall": recall, "accuracy": accuracy, "cross_entropy": cross_entropy,
        "seed": seed,
        "circuit": circuit.__name__,
        "learning_rate": lr,
        "loss_history": losses,
        "val_loss_history": val_losses,
        "val_f1_history": val_f1s,
        "avg_epoch_time_seconds": avg_epoch_time,
        "network_str": str(model),
        "model_path": model_path,
        "epochs": EPOCHS,
        "n_qubits": N_QUBITS,
        "features_per_qubit": FEAT_PER_QUBIT,
        "scaler" : str(SCALER).replace("()", "")
    }
    print(f"\n{GREEN_TEXT}RESULTS:\n F1-Score:{f1}, Precision:{precision}, Recall:{recall}, Accuracy:{accuracy}, Loss:{cross_entropy} {RESET_COLOR} \n\n")
    
    # append the results_dict to the RESULTS_DF
    RESULTS_DF = pd.concat([RESULTS_DF, pd.DataFrame([results_dict])], ignore_index=True)
    RESULTS_DF.to_csv(path_or_buf=RESULTS_PATH) # save for each run (in case of a crash)

    id += 1

RUN 1:
 DoubleEntanglementVLCircuit 
 lr: 0.001


Training progress: 100%|██████████| 304/304 [03:47<00:00,  1.34batch/s]


Epoch 1/10, Train-Loss: 0.5668, Val-Loss: 0.3372, Val-F1: 0.9818, Val-Precision: 0.9763, Val-Recall: 0.9872, Val-Accuracy: 0.9811


Training progress: 100%|██████████| 304/304 [03:38<00:00,  1.39batch/s]


Epoch 2/10, Train-Loss: 0.2283, Val-Loss: 0.1572, Val-F1: 0.9912, Val-Precision: 0.9952, Val-Recall: 0.9872, Val-Accuracy: 0.9910


...

Epoch 8/10, Train-Loss: 0.0359, Val-Loss: 0.0351, Val-F1: 0.9936, Val-Precision: 0.9952, Val-Recall: 0.9920, Val-Accuracy: 0.9934


Training progress: 100%|██████████| 304/304 [03:44<00:00,  1.36batch/s]


Epoch 9/10, Train-Loss: 0.0305, Val-Loss: 0.0313, Val-F1: 0.9936, Val-Precision: 0.9952, Val-Recall: 0.9920, Val-Accuracy: 0.9934


Training progress: 100%|██████████| 304/304 [03:44<00:00,  1.35batch/s]


Epoch 10/10, Train-Loss: 0.0268, Val-Loss: 0.0288, Val-F1: 0.9936, Val-Precision: 0.9952, Val-Recall: 0.9920, Val-Accuracy: 0.9934

RESULTS:
 F1-Score:0.9927826784282278, Precision:0.9967793880837359, Recall:0.9888178913738019, Accuracy:0.9926108374384236, Loss:0.02761727198958397  




In [7]:
RESULTS_DF

,id,f1_score,precision,recall,accuracy,cross_entropy,seed,circuit,learning_rate,loss_history,val_loss_history,val_f1_history,avg_epoch_time_seconds,network_str,model_path,epochs,n_qubits,features_per_qubit,scaler
0,0,0.992783,0.996779,0.988818,0.992611,0.027617,6,DoubleEntanglementVLCircuit,0.001,"[0.5668240442479912, 0.228333259540561, 0.1221...","[0.3371672034263611, 0.15724627673625946, 0.09...","[0.9817605075337034, 0.9911929543634908, 0.994...",222.027834,BinQuantumNeuralNetwork(\n (quantum_circuit_n...,results/models/bin/DoubleEntanglementVLCircuit...,10,4,4,MinMaxScaler
